In [ ]:
# from serial.main import train_maddpg
from parallel.main import train_maddpg
# from contract.main import run_cnp

min_scale = 5
middle_scale = 10
max_scale = 20

train_maddpg(middle_scale, episodes=1000)
# run_cnp(scale=middle_scale, episodes=50)

--- CNP Solver Ready ---
Ep 0 | Avg Reward: 0.22 | Success Rate: 0.72 | Time: 0.01s
Ep 1 finished after 9 steps.
Ep 1 | Avg Reward: 0.59 | Success Rate: 0.82 | Time: 0.00s
Ep 2 | Avg Reward: 0.13 | Success Rate: 0.75 | Time: 0.01s
Ep 3 finished after 13 steps.
Ep 3 | Avg Reward: -0.02 | Success Rate: 0.72 | Time: 0.01s
Ep 4 finished after 9 steps.
Ep 4 | Avg Reward: 0.36 | Success Rate: 0.83 | Time: 0.00s
Ep 5 finished after 13 steps.
Ep 5 | Avg Reward: 0.36 | Success Rate: 0.72 | Time: 0.01s
Ep 6 finished after 12 steps.
Ep 6 | Avg Reward: 0.31 | Success Rate: 0.75 | Time: 0.01s
Ep 7 finished after 11 steps.
Ep 7 | Avg Reward: 0.22 | Success Rate: 0.67 | Time: 0.00s
Ep 8 finished after 11 steps.
Ep 8 | Avg Reward: 0.22 | Success Rate: 0.69 | Time: 0.01s
Ep 9 | Avg Reward: 0.28 | Success Rate: 0.83 | Time: 0.01s
Ep 10 finished after 9 steps.
Ep 10 | Avg Reward: 0.35 | Success Rate: 0.72 | Time: 0.00s
Ep 11 finished after 11 steps.
Ep 11 | Avg Reward: 0.48 | Success Rate: 0.83 | Time: 0